In [1]:
using LinearAlgebra, Polyhedra
using JuMP, Ipopt, CDDLib, Clarabel
using Random
using PhyloNetworks

In [2]:
# ====================================================
#                    POLYHEDRAL
# ====================================================

"""
Calculate the polyhedral distance between two vectors.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
"""
function poly_dist(vec1, vec2, alphas)
    differences = alphas * (vec1 - vec2)
    return maximum(differences)
end

"""
Calculate the sum of polyhedral distances between `ref` and the points in `sample`.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
"""
function sum_of_poly_dist(ref, sample, alphas; power=1)
    return sum([poly_dist(pt, ref, alphas)^power for pt in sample])
end

"""
Find one polyhedral Fréchet mean of a given sample.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
"""
function poly_frechet(sample, alphas; power=2)
    
    dim = length(sample[1])
    
    # Choose model depending on power
    if power == 1
        error("FW computation not yet implemeneted")
    elseif power == 2
        model = Model(Clarabel.Optimizer)
    else
        model = Model(Ipopt.Optimizer)
    end
    
    set_silent(model)   # suppress printing

    @variable(model, x[1:dim])
    @variable(model, t[1:length(sample)])

    @objective(model, Min, sum(t))
    
    progress = 0
    total = length(sample) * size(alphas, 1)

    for (p_idx, p) in enumerate(sample)
        expressions = alphas * (x - p)
        
        for expr in expressions
            progress += 1
            total = length(sample) * size(expressions, 1)
            @constraint(model, t[p_idx] >= (expr)^power)
        end
        
        print("Setting up constraints: $(round(progress/total * 100, digits=3))%   \r")
        flush(stdout)
    end
    
    println()
    println("Optimising...")
    
    optimize!(model)
    minimiser = value.(x)
    
    return minimiser
end

# MethodError: no method matching 
# poly_frechet_set(::Vector{Vector{Rational{Int64}}}, ::Matrix{Rational{Int64}}; power::Int64, rep::String, tol::Float64)

"""
Find the set of polyhedral Fréchet means of a given sample.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
`rep` is either "vrep" or "hrep" -- returning either vertices or halfspaces.
"""
function poly_frechet_set(sample, alphas; power=2, rep::String="vrep", tol=1e-3)
    
    dim = length(sample[1])
    num_facets = size(alphas)[1]
    
    # Compute one Fréchet mean
    one_mean = poly_frechet(sample, alphas, power=power)
    
    println("Frechet mean found: ", one_mean)
    
    # Rationalise all coordinates
    rat_alphas = rationalize.(alphas, tol=tol)
    rat_sample = [rationalize.(pt, tol=tol) for pt in sample]
    rat_mean = rationalize.(one_mean, tol=tol)
    
    distances = [poly_dist(rat_mean, pt, alphas) for pt in rat_sample]
    
    Amat = vcat(rat_alphas, -rat_alphas)
    bval1 = Rational{Int64}[]
    bval2 = Rational{Int64}[]
    
    # Discard redundant halfspacesprintln("Setting up constraints: $(round(progress/total * 100, digits=3))%   \r")
    evals = rat_alphas * hcat([rat_mean - pt for pt in rat_sample]...)
    for k in 1:length(rat_sample)
        evals[:,k] .-= distances[k]
    end
    
    progress = 0
    total = num_facets
    
    for k = 1:num_facets
        
        greatest_nonpos = argmax([x > 0 ? -Inf : x for x in evals[k,:]])
        push!(bval1, dot(rat_alphas[k,:], rat_sample[greatest_nonpos]) + 
            distances[greatest_nonpos])
        
        progress += 1
        print("Removing redundant half-spaces: $(round(progress/total * 100, digits=3))%   \r")
        flush(stdout)
    
    end
    
    println()
    println("Finding defining facets..")
    
    poly = polyhedron(hrep(rat_alphas, bval1), CDDLib.Library(:exact))
    removehredundancy!(poly)
    
    if rep == "hrep"
        return poly
    elseif rep == "vrep"
        println("Finding vertices...")
        return vrep(poly)
    else
        println("Defaulting to hrep.")
        return poly
    end
    
end

poly_frechet_set

In [26]:
# ====================================================
#                    TROPICAL
# ====================================================


"""
Find the relevant facet normals in n dimensions for a tropical ball
"""
function trop_facets(n::Int64)
    result = zeros(Rational{Int64}, n * (n - 1), n)
    k = 1
    for i = 1:n
        for j = 1:n
            if i != j
                result[k, i] = 1//1
                result[k, j] = -1//1
                k += 1
            end
        end
    end
    return result
end

"""
Calculate the tropical distance between two vectors.
"""
function trop_dist(vec1, vec2)
    return maximum(vec1 - vec2) - minimum(vec1 - vec2)
end

"""
Calculate the sum of tropical distances between `ref` and the points in `sample`.
"""
function sum_of_trop_dist(ref, sample; power=1)
    return sum([trop_dist(pt, ref)^power for pt in sample])
end

"""
Find one polyhedral Fréchet mean of a given sample.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
"""
function trop_frechet(sample; power=2)
    dim = length(sample[1])
    alphas = trop_facets(dim)
    return poly_frechet(sample, alphas, power=power)
end

"""
Find one polyhedral Fréchet mean of a given sample.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
"""
function trop_frechet_set(sample; power=2, rep::String="vrep", tol=1e-3)
    dim = length(sample[1])
    alphas = trop_facets(dim)
    return poly_frechet_set(sample, alphas, power=power, rep=rep, tol=tol)
end

trop_frechet_set

In [27]:
sample = [[0,0,0], [0,4,1]]

my_point = [7, 10, 2]

println("Tropical distances: ", 
    [trop_dist(pt, my_point) for pt in sample])

println("Sum of tropical distances: ", 
    sum_of_trop_dist(my_point, sample))

println("Sum of squared tropical distances: ", 
    sum_of_trop_dist(my_point, sample, power=2))

trop_frechet_set(sample, rep="vrep", tol=1e-3)

Tropical distances: [8, 6]
Sum of tropical distances: 14
Sum of squared tropical distances: 100
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.8409270040529621, 1.1591895319222978, -0.12127076301825977]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Finding vertices...


V-representation CDDGeneratorMatrix{Rational{BigInt}, GMPRational}:
2-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[-1, 1, 0]
 Rational{BigInt}[0, 2, 0],
1-element iterator of Line{Rational{BigInt}, Vector{Rational{BigInt}}}:
 Line(Rational{BigInt}[1, 1, 1])

In [28]:
### Facets for the L-infinity norm
### Technically twice as long as strictly necessary
Linf_facets = [ 1  0 ;
               -1  0 ;
                0  1 ;
                0 -1 ]

sample = [ [0, 0], 
           [2, 2] ]

my_point = [7, 10]

println("Polyhedral distances: ", 
    [poly_dist(pt, my_point, Linf_facets) for pt in sample])

println("Sum of polyhedral distances: ", 
    sum_of_poly_dist(my_point, sample, Linf_facets))

println("Sum of squared polyhedral distances: ", 
    sum_of_poly_dist(my_point, sample, Linf_facets, power=2))

poly_frechet_set(sample, Linf_facets, rep="vrep", tol=1e-3)

Polyhedral distances: [10, 8]
Sum of polyhedral distances: 18
Sum of squared polyhedral distances: 164
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [1.0000522055841412, 1.0000522055832983]
Removing redundant half-spaces: 100.0%   
Finding defining facets..
Finding vertices...


V-representation CDDGeneratorMatrix{Rational{BigInt}, GMPRational}:
1-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[1, 1]

In [29]:
sample = [[-3,0,0], [0,-6,0], [0,0,-12]]

trop_frechet_set(sample, rep="vrep", tol=1e-3)

Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.0986940337952343, -0.09869403509820476, -1.0987180960744327]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Finding vertices...


V-representation CDDGeneratorMatrix{Rational{BigInt}, GMPRational}:
1-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[1, 1, 0],
1-element iterator of Line{Rational{BigInt}, Vector{Rational{BigInt}}}:
 Line(Rational{BigInt}[1, 1, 1])

In [30]:
sample = [[-3,0,0], [0,-6,0], [0,0,-12]]

trop_frechet_set(sample, rep="vrep", tol=1e-3)

Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.0986940337952343, -0.09869403509820476, -1.0987180960744327]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Finding vertices...


V-representation CDDGeneratorMatrix{Rational{BigInt}, GMPRational}:
1-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[1, 1, 0],
1-element iterator of Line{Rational{BigInt}, Vector{Rational{BigInt}}}:
 Line(Rational{BigInt}[1, 1, 1])

In [31]:
n = 20
m = n
rand_sample = [rand(1:20, n) for _ in 1:m]
trop_frechet_set(rand_sample, rep="vrep", tol=1e-3)

Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.24221893617879897, -0.242219153916155, -0.24221919999915578, -0.3100743456496952, 0.7577808512677587, -0.24221917085462394, 0.7577808269320436, -0.24221902499015774, 0.7577808054229284, -0.24221950796644948, -0.242219193613442, 0.09111480752314158, 0.30111405103962907, -1.4736784055245447, 1.7577808023816406, -1.2422192316823872, -1.5040541971628987, -1.2422191613219045, -0.24221919942910886, 0.09111477764760317]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Finding vertices...


V-representation CDDGeneratorMatrix{Rational{BigInt}, GMPRational}:
16-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[-106//319, -106//319, -106//319, -425//319, 213//319, -106//319, 213//319, -106//319, 213//319, -106//319, -106//319, 0, 213//319, -744//319, 532//319, -425//319, -425//319, -425//319, -106//319, 0]
 Rational{BigInt}[-106//319, -106//319, -106//319, 213//319, 213//319, -106//319, 213//319, -106//319, 213//319, -106//319, -106//319, 0, 213//319, -744//319, 532//319, -425//319, -425//319, -425//319, -106//319, 0]
 Rational{BigInt}[-106//319, -106//319, -106//319, -425//319, 213//319, -106//319, 213//319, -106//319, 213//319, -106//319, -106//319, 0, -106//319, -744//319, 532//319, -425//319, -425//319, -425//319, -106//319, 0]
 Rational{BigInt}[-106//319, -106//319, -106//319, 213//319, 213//319, -106//319, 213//319, -106//319, 213//319, -106//319, -106//319, 0, -106//319, -744//319, 532//319, -425//319, -425//319, -425//319, -106//319, 0]
 Rational{BigIn

In [32]:
n = 3

Random.seed!(50)

for m = 2:3n
    dim_vec = zeros(Int64, n+1)
    for _ = 1:100
        rand_sample = [rand(1:10, n) for _ in 1:m]
        try
            my_frech = trop_frechet_set(rand_sample, rep="hrep", tol=1e-3)
            dim_vec[dim(my_frech) + 1] += 1
        catch e
            println(rand_sample)
        end
    end
    println(m, " sample points ", dim_vec / sum(dim_vec))
end

Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.7881074955664398, 1.7121342125396724, 1.712134104346032]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-4.0248959635987225, -4.363118732300793, 0.6368533994607306]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [2.2998290579093332, 2.7944163452423405, 8.799768948219752]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-11.962483377823625, -7.962483382954251, -13.962520673963914]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [2.9635786456047883, -1.0364214318651095, 2.4634410839552126]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [0.10105383807909114, 0.7677085815303646, -0.0656380346404926]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.5848178649307317, 1.415182135170373, 0.16518400754566998]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [0.055954784025423254, -0.08686310197351864, -0.08686313998015036]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [0.9959673143779165, 0.7736139234721703, -1.0040326408198528]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-2.922416355975468, -0.922416361397493, -2.922416358040

Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.7272991659293884, -0.06064793859624628, 3.2727008335074332]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.054280155290183936, -0.0542801549924251, -0.05428015554640743]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.3167473369040969, 0.058673595945513475, 0.3499357820122472]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [7.239675221175949, 4.572775803911175, 6.8485608978693095]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-2.3920238714381905, -0.058717495716262114, 1.105422353

Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [1.5757340209329487, -0.4241969371131083, 0.5758030616320067]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.43431788486658446, 0.56568210835844, 1.5655648121392318]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.7448215934073128, -2.7439463705543017, 2.2551779990939504]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [1.6101022771036415, 1.1102189530846747, 3.610102421016077]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [0.09446991327852812, 0.09446991314503911, -0.4055150751130097]


...

Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.3343420359945956, 0.3323316810740291, -3.6676683259318232]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.9277538533530106, -0.3277790411355074, -0.3277790353968894]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [0.2163855033839067, -0.7836144951385048, 1.049740543433162]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [1.5005721441708408, -1.070788744527738, -0.11845424998820951]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [0.3413462256813652, -0.658653744596819, 0.19849075208157552]
Re

Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [0.7744861395264959, 1.3627470423189785, -1.1666732011369487]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.06324348360880697, 0.603362986064427, -1.396637006117131]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.15954926804957334, 0.8404507437253257, -0.6595799240059618]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [0.19054516881130037, 1.523842502915471, -0.8094548309458381]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.4701616534694236, 0.5298384611695224, 1.8631698839097854]
Rem

Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [3.7795824352742007, 1.1129408664836091, 3.7131135378019793]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.0069408414380710505, 1.1359837298627917, 0.8502780226620644]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-0.024453246949640038, -0.12976249670310985, 0.3439343950424689]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-1.15689970001578, 0.1372192473198531, 0.6078048654922472]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
8 sample points [0.0, 1.0, 0.0, 0.0]
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [1.840210623041727, -0.15

Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.518075032960925, 0.1485691833676121, 0.14856919124366988]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.012677260512009843, 0.9873227370187787, -0.298369460949651]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [0.3764433024987372, 0.37644330216882055, -0.6235566970187093]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.11170205178295593, 0.6575144900334127, -0.4963188135682792]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-1.4446533134475468, 0.8161952640862715, 0.7727225790536

Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [1.4778346938404456, -0.5221653139087477, 0.477982713595241]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [0.4757814734549509, 0.9758333760206835, 1.475781473443719]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-1.3240094899383776, -0.21282724397804864, -1.768534642738961]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [1.796068837550227, 0.796068844982479, -0.20393116245959927]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [0.6545636293508591, 0.4045613739088502, -0.3454363698352337]
Re

In [33]:
using JSON3
using DataFrames

# Function to read the JSON file and convert to a list of matrices
function read_and_convert_json(file_path::String)
    # Read the JSON file
    json_data = JSON3.read(file_path)
    
    # Extract elements from the nested arrays
    elements = [x[1] for x in json_data]
    elements = rationalize.(10000 * elements, tol=1e-2)
    
    # Convert elements into matrices
    num_elements = length(elements)
    matrices = []
    
    for i in 1:64:num_elements
        # Get the next 64 elements
        matrix_elements = elements[i:min(i+63, num_elements)]
        
        # Convert to an 8x8 matrix if there are 64 elements, otherwise create a smaller matrix
        matrix_size = length(matrix_elements)
        sqrt_size = Int(sqrt(matrix_size))
        push!(matrices, reshape(matrix_elements, sqrt_size, sqrt_size))
    end
    
    return matrices
end

# Read and convert the JSON file
file_path = "all_matrices.json"
matrices = read_and_convert_json(file_path)
taxa = ["Tg", "Et", "Cp", "Ta", "Bb", "Tt", "Pv", "Pf"]

8-element Vector{String}:
 "Tg"
 "Et"
 "Cp"
 "Ta"
 "Bb"
 "Tt"
 "Pv"
 "Pf"

In [34]:
"""
Take a matrix of pairwise distances between taxa and returns the cophenetic vector.
"""
function cophenetic_from_distance(pairwise)
    n = size(pairwise, 1)
    coph = [pairwise[i, j] for i in 1:n-1 for j in i+1:n]
    return coph
end

"""
Check if a distance matrix defines a phylogenetic tree
"""
function is_phylogenetic_tree(D)
    n = size(D, 1)
    
    # Check if the matrix is symmetric and non-negative
    for i in 1:n
        for j in i:n
            if D[i, j] != D[j, i] || D[i, j] < 0
                return false
            end
        end
    end

    # Check the four-point condition
    for i in 1:n-3
        for j in i+1:n-2
            for k in j+1:n-1
                for l in k+1:n
                    # Calculate distances
                    D_ij_kl = D[i, j] + D[k, l]
                    D_ik_jl = D[i, k] + D[j, l]
                    D_il_jk = D[i, l] + D[j, k]
                    
                    # Check the four-point condition
                    if !(D_ij_kl >= D_ik_jl && D_ij_kl >= D_il_jk) &&
                       !(D_ik_jl >= D_ij_kl && D_ik_jl >= D_il_jk) &&
                       !(D_il_jk >= D_ij_kl && D_il_jk >= D_ik_jl)
                        return false
                    end
                end
            end
        end
    end
    
    return true
end

"""
Check if a distance matrix defines an ultrametric tree
"""
function is_ultrametric_tree(D)
    n = size(D, 1)
    
    # Check if the matrix is symmetric and non-negative
    for i in 1:n
        for j in i:n
            if D[i, j] != D[j, i] || D[i, j] < 0
                return false
            end
        end
    end

    # Check the ultrametric condition
    for i in 1:n-2
        for j in i+1:n-1
            for k in j+1:n
                # Calculate distances
                Dij = D[i, j]
                Dik = D[i, k]
                Djk = D[j, k]
                
                # Check if the largest distance is attained at least twice
                if !(Dij <= max(Dik, Djk) && Dik <= max(Dij, Djk) && Djk <= max(Dij, Dik))
                    return false
                end
            end
        end
    end
    
    return true

end

is_ultrametric_tree

In [35]:
println("All phylogenetic: ", all([is_phylogenetic_tree(matrices[i]) for i = 1:268]))
println("All ultrametric: ", all([is_ultrametric_tree(matrices[i]) for i = 1:268]))

All phylogenetic: true
All ultrametric: false


In [36]:
coph_vecs = [cophenetic_from_distance(mat) for mat in matrices]
phylo_frech = trop_frechet_set(coph_vecs, rep="hrep")

Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-0.0015867523627228015, -0.0015858596202959394, -0.001587104964595801, -0.0015858506181254612, -0.0015865876895483797, -0.0015864941908716938, -0.0015863884219746226, -0.0015876505377054623, -0.0015895436423822382, -0.0015884260106528436, -0.0015892427482581753, -0.0015890728805328716, -0.0015884348907476435, -0.0015877703613602233, -0.0015863026836494224, -0.0015870529355101757, -0.001587014221983614, -0.001587015161827661, -0.0015908647286117298, -0.001670828638105668, -0.0015939926477330558, -0.0015904144265132473, -0.001590727988903485, -0.0015895592540557083, -0.001588986744578592, -0.0015945093240258043, -0.0015929248978799335, -0.001592318537568093]
Removing redundant half-spaces: 100.0%    
Finding defining facets..


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
4-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 1//149382)
 HyperPlane(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 1//149382)
 HyperPlane(Rational{BigInt}[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 1//149382)
 HyperPlane(Rational{BigInt}[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 1//149382),
294-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1192//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 129364811//149382)
 HalfSpace(Rational{BigInt}[0, 1, 

In [37]:
dim(phylo_frech)

24

In [38]:
N = 10
M = 100
Random.seed!(123)

dimensions = [[] for _ = 3:N]
for n = 3:N
    for m = 2:M
        sample = [Rational.(rand(0:10000, n)) for _ in 1:m]
        push!(dimensions[n-2], dim(trop_frechet_set(sample, rep="hrep")))
        println("m = ", m, "; n = ", n, " done!")
    end
end

dimensions

Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [320812.86816979846, 322615.67646795744, 323460.61041728035]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
m = 2; n = 3 done!
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [6904.120027912113, 12791.412006837381, 12164.39638442639]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
m = 3; n = 3 done!
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-1.4682012426515868e8, -1.4682113753115863e8, -1.4682259879300803e8]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
m = 4; n = 3 done!
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [-173301.33880585208, -175344.12294710745, -172841.83206543836]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
m = 5; n = 3 done!
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-8.25909458810197e8, -8.259055918778244e8, -8.25908

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




Optimising...
Frechet mean found: [-1.8643496624483985e8, -1.8643437789541438e8, -1.8643508155364132e8, -1.8643434431112802e8]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
m = 44; n = 4 done!
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-43283.17479748926, -43840.48978091038, -42763.71842681311, -43426.338450271476]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
m = 45; n = 4 done!
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-2822.9625009743404, -2696.590319632777, -1972.6529258556648, -2188.771806189955]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
m = 46; n = 4 done!
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [3.130064968056372e7, 3.1300564844357856e7, 3.130087947952792e7, 3.130028727461228e7]
Removing redundant half-spaces: 100.0%    
Finding defining facets..
m = 47; n = 4 done!
Setting up constraints: 100.0%    
Optimising...
Frechet m

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




Optimising...
Frechet mean found: [1.957805279326055e8, 1.9577999364786553e8, 1.957795627781852e8, 1.9577935813527417e8, 1.9578006367340833e8]
Removing redundant half-spaces: 100.0%   
Finding defining facets..
m = 17; n = 5 done!
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [12467.461799641604, 10678.872811406764, 11602.849616648984, 10948.589493539805, 11065.569877252898]
Removing redundant half-spaces: 100.0%   
Finding defining facets..
m = 18; n = 5 done!
Setting up constraints: 100.0%    
Optimising...
Frechet mean found: [-85589.88000182336, -86662.35121312729, -86313.27087465713, -87306.40633196711, -87330.28964293975]
Removing redundant half-spaces: 100.0%   
Finding defining facets..
m = 19; n = 5 done!
Setting up constraints: 100.0%   
Optimising...
Frechet mean found: [62855.14112810575, 63398.927815709874, 64098.718543004215, 63856.9397586193, 63085.13186146823]
Removing redundant half-spaces: 100.0%   
Finding defining facets..
m = 20; n = 5 done!

Excessive output truncated after 524304 bytes.

8-element Vector{Vector{Any}}:
 [2, 2, 2, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [3, 2, 2, 2, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [4, 3, 4, 3, 2, 1, 2, 1, 2, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [5, 5, 3, 3, 3, 3, 1, 2, 2, 3  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [7, 7, 6, 3, 3, 4, 2, 4, 4, 4  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [7, 8, 5, 4, 6, 4, 6, 6, 2, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [8, 7, 8, 7, 6, 5, 6, 3, 3, 4  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [9, 9, 10, 6, 7, 6, 4, 5, 5, 6  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [39]:
using Plots

# Example data: categories and their respective proportions in different groups
categories = ["Category 1", "Category 2", "Category 3"]

proportional_dimensions = [dim_list / M for dim_list in dimensions]
data = hcat(proportional_dimensions...)

# Create the proportional bar chart
bar(data, label=1:M, legend=:topright, title="Proportional Bar Chart", xlabel="Groups", ylabel="Proportion", bar_width=0.7, lw=0, series_annotations=transpose([group1 group2 group3]))

# Save the plot to a file
savefig("proportional_bar_chart.png")

# Display the plot
display(plot)

[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a]


LoadError: UndefVarError: `group1` not defined

In [40]:
for thing in dimensions
    println(thing)
    println()
end

Any[2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Any[3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Any[4, 3, 4, 3, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Any[5, 5, 3, 3, 3, 3, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 